# Azure ML Workspace

## Importing libraries

In [78]:
from azureml.core import Workspace, Environment, Model, Dataset
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

## Initialise workspace
Initialise a workspace using the values from config.json
If this file is not found, an exception occurs
When an exception is detected, a workspace using the defined details is created

In [79]:
try:
    ## Try to load workspace
    wspace = Workspace.from_config()
except Exception as ex:
    ## Create new workspace if existing is not found
    wspace = Workspace.create(name = "music-genre-classification",
                                  subscription_id = "11636c4c-f408-4202-980a-b738e97f4d7f",
                                  resource_group = "classification-ml",
                                  create_resource_group = True,
                                  location = "uksouth")

    ## Write workspace config to a local file
    wspace.write_config()

print("Workspace successfully loaded")

Workspace successfully loaded


## Create an environment
Create an environment for the model to run on the Azure servers

In [80]:
##  Load conda dependencies required for this environment.
##  Register environment for the workspace
env = Environment("env")

conda_dep = CondaDependencies.create(pip_packages = ["pandas == 1.1.3",
                                                    "azureml-core",
                                                    "azureml-defaults >= 1.0.45"],
                                    conda_packages = ["scikit-learn == 0.23.2"])

env.python.conda_dependencies = conda_dep

env.register(workspace = wspace)

print("Environment Registered")

## Get the environment created from the workspace and create a file to verify it has been defined
my_env = Environment.get(workspace = wspace, name = "env")
my_env.save_to_directory("./environ", overwrite = True)

print("Environment Saved")

inference_config = InferenceConfig(entry_script = "score.py",
                                   environment = my_env)



Environment Registered
Environment Saved


## Register the model
Register the saved model created by "knn-model-tuning" for use on Azure ML

In [81]:
model = Model.register(workspace = wspace,
                       model_name = "knn",
                       model_path = "models/knn.pkl",
                       model_framework = "ScikitLearn",
                       description = "Tuned KNN for Genre Classification")

Registering model knn


## Set up Azure Container Instance
Define ACI Webservice

In [82]:
aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1,
    memory_gb = 1)

## Upload dataset
Upload the dataset for user with models

In [83]:
# datastore = wspace.get_default_datastore()
# datastore.upload_files(files = ["extracted-features.csv"],
#                        target_path = "data")
# dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, "data/extracted-features.csv")])
#
# dataset = dataset.register(workspace = wspace,
#                                  name = "GTZAN Extracted Features",
#                                  description = "Features extracted from the GTZAN dataset")

## Deploy the model
Deploy the model onto the defined ACI container using the set environment, inference config and workspace

In [84]:
service = Model.deploy(workspace = wspace,
                       name = "music-genre-classification",
                       models = [model],
                       inference_config = inference_config,
                       deployment_config = aci_config,
                       overwrite = True)

service.wait_for_deployment(show_output = True)
print(service.get_logs())

print(service.scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running....
Succeeded
ACI service creation operation finished, operation "Succeeded"
2021-04-14T10:38:31,788320371+00:00 - nginx/run 
2021-04-14T10:38:31,785362883+00:00 - gunicorn/run 
/usr/sbin/nginx: /azureml-envs/azureml_d322234cf1103dbd7b07da4ab58adb31/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_d322234cf1103dbd7b07da4ab58adb31/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_d322234cf1103dbd7b07da4ab58adb31/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_d322234cf1103dbd7b07da4ab58adb31/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr